In [1]:
import pandas as pd
import geopandas as gpd
import os
import requests
import time
import io

DATA_DIR = '../data/raw/'
INTERIM_DIR = '../data/interim/'
CTA_FILE_PATH = os.path.join(DATA_DIR, 'cta_l_stops.geojson')
os.makedirs(INTERIM_DIR, exist_ok=True)

In [2]:
gdf_cta = gpd.read_file(CTA_FILE_PATH)
gdf_cta

,station_id,point_y,pknrd,legend,point_x,address,ada,longname,lines,geometry
0,970,1896352.78926296,False,Blue Line,1144440.97667316,720 S. Cicero Avenue,False,Cicero-Congress,Blue Line (Congress),POINT (-87.74517 41.87161)
1,20,1901803.39560403,False,Green Line,1128608.76033842,1 S. Harlem Avenue,True,Harlem-Lake,Green Line (Lake),POINT (-87.80318 41.88685)
2,610,1901950.19138017,False,Green Line,1133921.86037537,36 N. Ridgeland Avenue,False,Ridgeland,Green Line (Lake),POINT (-87.78366 41.88716)
3,230,1937256.04587705,True,Blue Line,1118914.13069739,5800 N. Cumberland Avenue,True,Cumberland,Blue Line,POINT (-87.83803 41.98429)
4,1700,1900828.68763513,False,Multiple Lines,1176812.47724731,29 N. Wabash,True,Washington/Wabash,"Brown, Orange, Pink, Purple (Express), Green",POINT (-87.62619 41.88322)
...,...,...,...,...,...,...,...,...,...,...
140,330,1903902.00526999,False,Red Line,1176288.17926282,521 N. State Street,True,Grand/State,Red Line,POINT (-87.62802 41.89167)
141,1040,1890060.7219291,False,Pink Line,1155312.29879529,1944 S. Kedzie Avenue,True,Kedzie-Douglas,Pink Line,POINT (-87.70543 41.85413)
142,1670,1901252.71835552,False,Green Line,1152209.0952084,3631 W. Lake Street,True,Conservatory-Central Park,Green Line (Lake),POINT (-87.71652 41.8849)
143,600,1889868.06298492,False,Pink Line,1147729.86397223,2019 S. Kostner Avenue,True,Kostner,Pink Line,POINT (-87.73326 41.85375)


In [3]:
print("Initial CTA Station Columns:", gdf_cta.columns.tolist())
gdf_cta

Initial CTA Station Columns: ['station_id', 'point_y', 'pknrd', 'legend', 'point_x', 'address', 'ada', 'longname', 'lines', 'geometry']


,station_id,point_y,pknrd,legend,point_x,address,ada,longname,lines,geometry
0,970,1896352.78926296,False,Blue Line,1144440.97667316,720 S. Cicero Avenue,False,Cicero-Congress,Blue Line (Congress),POINT (-87.74517 41.87161)
1,20,1901803.39560403,False,Green Line,1128608.76033842,1 S. Harlem Avenue,True,Harlem-Lake,Green Line (Lake),POINT (-87.80318 41.88685)
2,610,1901950.19138017,False,Green Line,1133921.86037537,36 N. Ridgeland Avenue,False,Ridgeland,Green Line (Lake),POINT (-87.78366 41.88716)
3,230,1937256.04587705,True,Blue Line,1118914.13069739,5800 N. Cumberland Avenue,True,Cumberland,Blue Line,POINT (-87.83803 41.98429)
4,1700,1900828.68763513,False,Multiple Lines,1176812.47724731,29 N. Wabash,True,Washington/Wabash,"Brown, Orange, Pink, Purple (Express), Green",POINT (-87.62619 41.88322)
...,...,...,...,...,...,...,...,...,...,...
140,330,1903902.00526999,False,Red Line,1176288.17926282,521 N. State Street,True,Grand/State,Red Line,POINT (-87.62802 41.89167)
141,1040,1890060.7219291,False,Pink Line,1155312.29879529,1944 S. Kedzie Avenue,True,Kedzie-Douglas,Pink Line,POINT (-87.70543 41.85413)
142,1670,1901252.71835552,False,Green Line,1152209.0952084,3631 W. Lake Street,True,Conservatory-Central Park,Green Line (Lake),POINT (-87.71652 41.8849)
143,600,1889868.06298492,False,Pink Line,1147729.86397223,2019 S. Kostner Avenue,True,Kostner,Pink Line,POINT (-87.73326 41.85375)


In [4]:
name_column = 'longname' 
lines_column = 'legend'
gdf_cta['Line_Colors'] = gdf_cta[lines_column].str.replace(' Line', '', regex=False).str.replace(', ', ',', regex=False)
gdf_cta[[name_column, lines_column, 'Line_Colors', 'geometry']].head()

,longname,legend,Line_Colors,geometry
0,Cicero-Congress,Blue Line,Blue,POINT (-87.74517 41.87161)
1,Harlem-Lake,Green Line,Green,POINT (-87.80318 41.88685)
2,Ridgeland,Green Line,Green,POINT (-87.78366 41.88716)
3,Cumberland,Blue Line,Blue,POINT (-87.83803 41.98429)
4,Washington/Wabash,Multiple Lines,Multiples,POINT (-87.62619 41.88322)


In [5]:
print(f"Total unique station names: {gdf_cta[name_column].nunique()}")
print(f"Total rows in DataFrame: {len(gdf_cta)}")
print("Top 5 unique Line_Colors combinations:")
gdf_cta['Line_Colors'].value_counts().head(10)

Total unique station names: 145
Total rows in DataFrame: 145
Top 5 unique Line_Colors combinations:


Line_Colors
Blue         32
Multiples    30
Red          29
Green        23
Brown        11
Pink         11
Orange        7
Yellow        2
Name: count, dtype: int64

In [6]:
df_sales = pd.read_csv(os.path.join(DATA_DIR, 'sales_data.csv'))
df_sales.head()

/var/folders/fk/9chv4jr11ln50p4dhsxntpcw0000gn/T/ipykernel_44566/2852610417.py:1: DtypeWarning: Columns (8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sales = pd.read_csv(os.path.join(DATA_DIR, 'sales_data.csv'))


,pin,year,township_code,nbhd,class,sale_date,is_mydec_date,sale_price,doc_no,deed_type,mydec_deed_type,seller_name,is_multisale,num_parcels_sale,buyer_name,sale_type,sale_filter_same_sale_within_365,sale_filter_less_than_10k,sale_filter_deed_type,row_id
0,31012140340000,2000.0,32,32050,278,2000-04-01T00:00:00.000,False,177500.0,317676,Trustee,NaN,NaN,False,1.0,NaN,LAND AND BUILDING,False,False,False,96289215
1,14291030261014,2000.0,73,73150,299,2000-02-01T00:00:00.000,False,315000.0,326770,Warranty,NaN,NaN,False,1.0,NaN,LAND AND BUILDING,False,False,False,97142869
2,13164060180000,2000.0,71,71101,203,2000-06-01T00:00:00.000,False,192000.0,519440,Warranty,NaN,NaN,False,1.0,NaN,LAND AND BUILDING,False,False,False,96585660
3,24233020370000,2014.0,39,39250,100,2014-06-01T00:00:00.000,False,500.0,1427529079,Other,NaN,US BANK,False,1.0,MY OWN DOMINION LP,LAND,False,True,False,96639542
4,19354040600000,2016.0,72,72200,205,2016-08-01T00:00:00.000,False,1.0,1625129009,Warranty,NaN,NaN,True,5.0,NaN,LAND AND BUILDING,True,True,False,98154864


In [7]:
UNIVERSE_DATA_ID = 'nj4t-kc8j'
UNIVERSE_API_URL = f'https://datacatalog.cookcountyil.gov/resource/{UNIVERSE_DATA_ID}.csv'
TEXT_FILE = os.path.join(INTERIM_DIR, 'universe_pin_locations.txt')

In [8]:
# Clean TEXT_FILE by removing lines that contain "None,None" and trimming empty/trailing spaces
if os.path.exists(TEXT_FILE):
    with open(TEXT_FILE, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    before = len(lines)
    cleaned = []
    for line in lines:
        s = line.strip()
        if not s:
            continue
        if 'None,None' in s:
            continue
        cleaned.append(s)

    with open(TEXT_FILE, 'w', encoding='utf-8') as f:
        for ln in cleaned:
            f.write(f"{ln}\n")

    print(f"Cleaned {TEXT_FILE}: {before} -> {len(cleaned)} lines")
else:
    print(f"File not found: {TEXT_FILE}")

Cleaned ../data/interim/universe_pin_locations.txt: 537285 -> 537285 lines


In [9]:
# use existing cleaned lines if available, otherwise read from TEXT_FILE
if 'cleaned' in globals():
    src_lines = cleaned
else:
    with open(TEXT_FILE, 'r', encoding='utf-8') as f:
        src_lines = [ln.strip() for ln in f if ln.strip()]

unique_pin10 = set()
for ln in src_lines:
    # skip header or empty lines
    if not ln or ln.lower().startswith('pin10'):
        continue
    parts = ln.split(',')
    if not parts:
        continue
    pin_raw = parts[0].strip()
    if not pin_raw or pin_raw.lower() == 'none':
        continue
    # normalize trailing .0 (e.g. "2503106015.0" -> "2503106015")
    if pin_raw.endswith('.0'):
        pin = pin_raw[:-2]
    else:
        pin = pin_raw
    unique_pin10.add(pin)

# result available as `unique_pin10`
print("Unique pin10 count:", len(unique_pin10))

Unique pin10 count: 532688


In [10]:
df_sales['pin10'] = df_sales['pin'].astype(str).str[:10]
all_pins = df_sales['pin10'].dropna().unique().tolist()

In [11]:
# compute pins_to_check = all_pins that are not present in unique_pin10 (preserve original order)
pins_to_check = [p for p in all_pins if p not in unique_pin10]

print(f"Total all_pins: {len(all_pins)}")
print(f"Pins already found (unique_pin10): {len(unique_pin10)}")
print(f"Missing pins_to_check count: {len(pins_to_check)}")

Total all_pins: 930661
Pins already found (unique_pin10): 532688
Missing pins_to_check count: 397973


In [12]:
PINS_TO_CHECK_FILE = os.path.join(INTERIM_DIR, 'pins_to_check.txt')
# # write one pin per line
# with open(PINS_TO_CHECK_FILE, 'w', encoding='utf-8') as fh:
#     for pin in pins_to_check:
#         fh.write(f"{pin}\n")

In [13]:
def fetch_pin(pin10):
    params = {
        '$limit': 1,
        '$select': 'pin10, lon, lat',
        '$where': f"pin10 = '{pin10}'"
    }
    try:
        r = requests.get(UNIVERSE_API_URL, params=params, timeout=10)
        r.raise_for_status()
        df = pd.read_csv(io.StringIO(r.text))
        if not df.empty:
            row = df.iloc[0]
            return {'pin10': str(row.get('pin10')), 'lon': row.get('lon'), 'lat': row.get('lat')}
    except Exception:
        return None
    return None

In [ ]:
import random
for each_pin in pins_to_check:
    result = fetch_pin(each_pin)
    if result:
        with open(TEXT_FILE, 'a', encoding='utf-8') as fh:
            fh.write(f"{result['pin10']},{result['lon']},{result['lat']}\n")
    pins_to_check.remove(each_pin)
    if random.randint(0, 30) == 0:
        os.remove(PINS_TO_CHECK_FILE)
        with open(PINS_TO_CHECK_FILE, 'w', encoding='utf-8') as fh:
            for pin in pins_to_check:
                fh.write(f"{pin}\n")